# **Guía de Laboratorio N°09: Inferencia de Redes de Regulación Genética**

> **Asignatura:**
Bioinformática

> **Docente:**
Carlos Fernando Montoya Cubas

> **Estudiante:**
Gerald Antonio Cusacani Gonzales (Cód. 200332)

## **Importación de Librerías**

In [1]:
# -- Importación de librerías necesarias para la definición de los algortimos
import numpy as np
import itertools
from collections import Counter
import random

## **Implementación de Algortimos**

In [2]:
def calculate_frequencies(data, target_gene, predictors):
    """
    Calcula una tabla de frecuencias para las combinaciones de valores de predictores
    en relación con los valores futuros de un gen objetivo.

    Entrada:
    - data: Lista de listas o matriz (número de tiempos x número de genes) que contiene
      los datos de expresión genética booleana.
    - target_gene: Índice del gen objetivo (columna en la matriz `data`).
    - predictors: Lista de índices de los genes predictores (columnas en la matriz `data`).

    Salida:
    - freq_table: Diccionario que contiene como claves las combinaciones posibles de valores
      de los predictores (tuplas) y como valores listas de tamaño 2, donde el índice 0
      corresponde a la frecuencia de 0 en el gen objetivo y el índice 1 a la frecuencia de 1.
    """
    # Número de predictores
    num_predictors = predictors.size

    # Generar todas las combinaciones posibles de valores para los predictores
    combinations = itertools.product([0, 1], repeat=num_predictors)

    # Inicializar la tabla de frecuencias
    freq_table = {tuple(comb): [0, 0] for comb in combinations}

    # Extraer las columnas de los predictores y el gen objetivo
    predictor_columns = data[:, predictors]
    target_column = data[:, target_gene]

    # Número de instancias en la matriz de datos
    num_times = data.shape[0]

    # Recorrer los datos para llenar la tabla de frecuencias
    for i in range(num_times - 1):
        # La clave es el estado actual de los predictores
        clave = tuple(predictor_columns[i])
        # El valor es el estado siguiente del gen objetivo
        valor = target_column[i + 1]
        # Incrementar el contador correspondiente en la tabla de frecuencias
        freq_table[clave][valor] += 1

    return freq_table


def evaluate_criterion(frequency_table):
    """
    Evalúa la función criterio |f(y=0) - f(y=1)| en base a la tabla de
    frecuencias.

    Entrada:
    - frequency_table: Tabla de frecuencias generada para una combinación
    de predictores.

    Salida:
    - quality_score: Puntuación numérica que representa la calidad de los
    predictores.
    """
    quality_score = 0

    # Recorrer la tabla de frecuencias
    for key, counts in frequency_table.items():
        f_y0, f_y1 = counts                 # Frecuencias para y = 0 e y = 1
        quality_score += abs(f_y0 - f_y1)   # Calcular y acumular

    return quality_score

def exhaustive_search(data, target_gene, max_dim=3, criterion_function=evaluate_criterion):
    """
    Realiza una búsqueda exhaustiva incremental para identificar el mejor subconjunto de predictores.

    Entrada:
    - data: Matriz (número de tiempos x número de genes) con datos de expresión genética.
    - target_gene: Índice del gen objetivo (columna en la matriz `data`).
    - max_dim: Tamaño máximo del subconjunto de predictores.
    - criterion_function: Función criterio para evaluar la calidad de los subconjuntos.

    Salida:
    - best_subsets: Lista de subconjuntos con la mejor puntuación.
    - best_score: La mejor puntuación alcanzada.
    """
    num_genes = data.shape[1]
    best_subsets = []
    best_score = 1  # Por definición, el conjunto vacío tiene una puntuación de 1

    for dim in range(1, max_dim + 1):
        current_best_score = 0
        current_best_subsets = []

        # Generar todos los subconjuntos de tamaño dim
        for subset in itertools.combinations(range(num_genes), dim):

            # Calcular la tabla de frecuencias para el subconjunto
            freq_table = calculate_frequencies(data, target_gene, np.array(subset))

            # Evaluar la calidad del subconjunto usando la función criterio
            score = criterion_function(freq_table)

            # Actualizar la mejor puntuación y subconjuntos para esta dimensión
            if score > current_best_score:
                current_best_score = score
                current_best_subsets = [subset]
            elif score == current_best_score:
                current_best_subsets.append(subset)

        # Comparar con la mejor puntuación de dimensiones anteriores
        if current_best_score <= best_score:
            break  # Detener la búsqueda si no se supera la mejor puntuación anterior

        # Actualizar los mejores resultados globales
        best_score = current_best_score
        best_subsets = current_best_subsets

    return random.choice(best_subsets), best_score

## **Implementación de clase**

In [3]:
class BooleanNetworkInference:
    """-------------------------------------------------------------------------
    Clase para implementar un algoritmo de selección de características
    para la inferencia en redes booleanas.
    -------------------------------------------------------------------------"""

    def __init__(self, file_path, search_function=None, criterion_function=None):
        """---------------------------------------------------------------------
        Inicializa la clase con la ruta del archivo, la función de búsqueda y
        la función criterio.

        Entrada:
        - file_path: Ruta al archivo .txt que contiene la matriz de expresión
        booleana.
        - search_function: Función que implementa el algoritmo de búsqueda de
        predictores.
        - criterion_function: Función que evalúa la calidad de los predictores.

        Salida:
        - Instancia inicializada con el conjunto de datos cargado, la función
        de búsqueda y la función criterio.
        ---------------------------------------------------------------------"""
        rows, cols, matrix = self.load_data_from_txt(file_path)
        self.num_times = rows
        self.num_genes = cols
        self.data = matrix
        self.search_function = search_function
        self.criterion_function = criterion_function
        self.network = None
        self.real_network = None

    def load_data_from_txt(self, file_path):
        """
        Carga un archivo .txt con la matriz de expresión genética y la convierte en una lista de listas.

        Entrada:
        - file_path: Ruta al archivo .txt que contiene la matriz de expresión booleana.
          La primera línea contiene las dimensiones de la matriz (tiempos x genes).
          Las siguientes líneas contienen los valores de la matriz en formato booleano.

        Salida:
        - list of list: Matriz de expresión genética representada como una lista de listas.
        """
        with open(file_path, 'r') as file:
            # Leer dimensiones de la matriz
            dimensions = file.readline().strip().split()
            rows, cols = int(dimensions[0]), int(dimensions[1])

            # Leer valores de la matriz
            matrix = np.array([list(map(int, line.strip().split())) for line in file.readlines()])

            # Verificar que las dimensiones especificadas coincidan con los datos
            if matrix.shape != (rows, cols):
                raise ValueError("Las dimensiones especificadas no coinciden con los datos en el archivo.")

        return rows, cols, matrix

    def data_desc(self):
        """
        Descripción de los datos de expresión genética.
        """
        print(f"Número de tiempos: {self.num_times}")
        print(f"Número de genes: {self.num_genes}")
        print()
        print("Matriz de expresión genética:")
        np.set_printoptions(linewidth=150)
        for row in self.data:
            print(row)

    def network_desc(self):
        """
        Muestra las claves y valores de un diccionario en el que los valores son tuplas de dos elementos.
        """
        print("")
        print("Formato:")
        print("[*]  >>   Idnetificador del Gen  | Mejores Conjuntos Predictores  |  Calificación")
        print()
        print("-"*70)
        for clave, (valor1, valor2) in self.network.items():
          print(f">>   {clave}  |  {valor1}  |  {valor2}")
          print("-"*70)

    def real_network_desc(self):
        """
        Muestra las claves y valores de un diccionario en el que los valores son tuplas de dos elementos.
        """
        print("")
        print("Formato:")
        print("[*]  >>   Idnetificador del Gen  | Mejores Conjuntos Predictores  |  Calificación")
        print()
        print("-"*70)
        for clave, (valor1, valor2) in self.real_network.items():
          print(f">>   {clave}  |  {valor1}  |  {valor2}")
          print("-"*70)

    def infer_network(self):
        """
        Infiera la red booleana aplicando el algoritmo a todos los genes como objetivos.

        Entrada:
        - Ninguna.

        Salida:
        - network: Diccionario donde las claves son genes objetivos y los
          valores son los mejores predictores respectivament y la calificación respectiva.
        """
        if self.search_function is None or self.criterion_function is None:
            raise ValueError("Se debe proporcionar una función de búsqueda y una función criterio.")

        network = {}
        # Iterar sobre cada gen como objetivo
        for target_gene in range(self.num_genes):
            # Ejecutar la función de búsqueda con el criterio para determinar el mejor predictor
            best_subset, best_score = self.search_function(
                data=self.data,
                target_gene=target_gene,
                max_dim=3,
                criterion_function=self.criterion_function
            )
            # Asociar el gen objetivo con su mejor predictor y la calificación respectiva
            network[target_gene] = (best_subset, best_score)

        # Actualizar estado de la red
        self.network = network

        return network

    def load_results_from_txt(self, file_path):
        """
        Carga los resultados reales desde un archivo .txt en el formato especificado.

        Entrada:
        - file_path: Ruta al archivo .txt que contiene los resultados reales.

        Salida:
        - results: Diccionario donde las claves son los índices de los genes,
          y los valores son tuplas con el mejor conjunto predictor (como una tupla)
          y la puntuación correspondiente.
        """
        results = {}

        with open(file_path, 'r') as file:
            lines = file.readlines()
            for index, line in enumerate(lines):
                if index == 0:
                    continue
                if line.strip():  # Ignorar líneas vacías
                    parts = line.strip().split('|')

                    # Procesar el mejor conjunto predictor
                    predictors_str = parts[0]
                    if predictors_str == "NN":  # Caso vacío
                        best_predictors = ()
                    else:
                        best_predictors = tuple(map(int, predictors_str.split(',')))

                    # Procesar la puntuación
                    best_score = int(parts[1])

                    # Agregar al diccionario
                    results[index-1] = (best_predictors, best_score)

        self.real_network = results
        return results

    def display_confusion_matrix(self, confusion_matrix):
        """
        Muestra la matriz de confusión en un formato legible.

        Entrada:
        - confusion_matrix: Diccionario con las categorías de evaluación:
          'Exacto', 'Parcial', 'Incorrecto'.
        """
        print("Matriz de Confusión")
        print("===================")
        for category, count in confusion_matrix.items():
            print(f"{category}: {count}")
        total = sum(confusion_matrix.values())
        print("===================")
        print(f"Total: {total}")
        if total > 0:
            print("===================")
            print("Porcentajes:")
            for category, count in confusion_matrix.items():
                percentage = (count / total) * 100
                print(f"{category}: {percentage:.2f}%")


    def generate_confusion_matrix(self):
        """
        Genera una matriz de confusión comparando los resultados inferidos con los reales.

        Entrada:
        - real_results: Diccionario donde las claves son genes, y los valores son
          tuplas con el mejor conjunto de predictores y la puntuación real.

        Salida:
        - confusion_matrix: Diccionario con las categorías de evaluación:
          'Exacto', 'Parcial', 'Incorrecto'.
        """
        margin = 0.2  # Margen de error del 20%
        confusion_matrix = {
            'Exacto': 0,
            'Parcial': 0,
            'Incorrecto': 0
        }

        for gene, (real_predictors, real_score) in self.real_network.items():
            inferred_predictors, inferred_score = self.network.get(gene, (None, None))

            # Evaluar coincidencia de conjuntos predictores
            predictors_match = set(real_predictors) == set(inferred_predictors)
            predictors_overlap = bool(set(real_predictors) & set(inferred_predictors))

            # Evaluar coincidencia de puntuación
            if real_score == 0:  # Caso especial: puntuación real igual a 0
                score_match = inferred_score == 0
            elif real_score is not None and inferred_score is not None:
                score_match = abs(real_score - inferred_score) / real_score <= margin
            else:
                score_match = False

            # Clasificar resultados
            if predictors_match and score_match:
                confusion_matrix['Exacto'] += 1
            elif predictors_overlap or score_match:
                confusion_matrix['Parcial'] += 1
            else:
                confusion_matrix['Incorrecto'] += 1

        # Mostrar matriz de confusión
        self.display_confusion_matrix(confusion_matrix)

## **Ejemplo Práctico (Guía de Laboratorio N°09)**

In [5]:


file_path = r"C:\Users\josep\OneDrive\Documentos\Semestre 2024-II\BioInformatica\expresion_genica.deg"

In [6]:
# -- Crear instancia de red
bni = BooleanNetworkInference(file_path, exhaustive_search, evaluate_criterion)

# -- Imprimir los datos cargados
bni.data_desc()

Número de tiempos: 20
Número de genes: 50

Matriz de expresión genética:
[0 1 1 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 1 1 0 1 1 1 0 0 0 0 1 0 1 1 0 1 1 0 0 0 1 1 1 1 0 1 1 0 1 1 0]
[0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1 0 0]
[0 1 1 1 1 0 1 1 0 0 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 1 1 1 0 1 0 0]
[1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 0 1 1 0 1 1 1 1 0 0 1 1 0 1 0 0 1 1]
[0 0 1 0 0 0 0 0 1 1 0 0 1 1 1 0 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 1 0 0 1 1 1 0 1 1 0 1 0 0 1 0 0 0 1]
[0 0 1 0 0 0 1 0 1 0 0 1 0 0 1 1 1 0 0 1 1 1 1 0 0 1 0 0 1 1 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 1 1 0 0 0]
[0 1 1 1 1 0 0 0 1 0 1 1 0 1 1 1 1 0 0 0 1 0 1 1 1 1 1 0 0 0 1 0 0 0 0 1 1 1 1 1 0 0 0 1 1 1 1 0 0 0]
[0 0 0 0 1 1 1 0 1 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 0 0 1 0 1 0 1 0 0 1]
[0 0 0 0 1 0 1 0 1 0 1 1 0 1 1 0 1 0 0 1 0 0 1 0 1 1 1 1 1 0 1 0 0 1 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 1]
[0 0 1 1 

In [7]:
# -- Aplicar el algortimo de selección de características
bni.infer_network()

# -- Mostrar resultado
bni.network_desc()


Formato:
[*]  >>   Idnetificador del Gen  | Mejores Conjuntos Predictores  |  Calificación

----------------------------------------------------------------------
>>   0  |  (4, 12)  |  19
----------------------------------------------------------------------
>>   1  |  (7,)  |  17
----------------------------------------------------------------------
>>   2  |  (20, 23, 26)  |  19
----------------------------------------------------------------------
>>   3  |  (9, 19)  |  19
----------------------------------------------------------------------
>>   4  |  (16, 46)  |  19
----------------------------------------------------------------------
>>   5  |  (30,)  |  19
----------------------------------------------------------------------
>>   6  |  (19, 28, 33)  |  19
----------------------------------------------------------------------
>>   7  |  (26, 41)  |  19
----------------------------------------------------------------------
>>   8  |  (4, 8, 28)  |  19
------------------------

In [ ]:
# Mostrar resultado reales
file_path = "/gdrive/MyDrive/RGC/bio_g9_result.txt"
bni.load_results_from_txt(file_path)
bni.real_network_desc()


Formato:
[*]  >>   Idnetificador del Gen  | Mejores Conjuntos Predictores  |  Calificación

----------------------------------------------------------------------
>>   0  |  ()  |  1
----------------------------------------------------------------------
>>   1  |  (13, 25)  |  9
----------------------------------------------------------------------
>>   2  |  (3, 8, 20, 24, 32)  |  907587710
----------------------------------------------------------------------
>>   3  |  (1,)  |  2
----------------------------------------------------------------------
>>   4  |  (0, 2, 26, 46)  |  47769
----------------------------------------------------------------------
>>   5  |  ()  |  0
----------------------------------------------------------------------
>>   6  |  (5, 25, 41)  |  138
----------------------------------------------------------------------
>>   7  |  (8, 23)  |  7
----------------------------------------------------------------------
>>   8  |  (13,)  |  2
---------------------

In [ ]:
# -- Generar la matriz de confución
bni.generate_confusion_matrix()

Matriz de Confusión
Exacto: 0
Parcial: 10
Incorrecto: 40
Total: 50
Porcentajes:
Exacto: 0.00%
Parcial: 20.00%
Incorrecto: 80.00%
